In [ ]:
import pandas as pd
from datetime import datetime

# --- User settings ---
# Modify the individual components of the start and end dates as needed.
start_year = 2025
start_month = 8
start_day = 7

end_year = 2025
end_month = 8
end_day = 8

# Define the names for your input and output files.
input_filename = 'datalog.txt'
output_filename = f"Datalog_{start_date_str.replace('-', '')}_{end_date_str.replace('-', '')}.csv"
# ---------------------


# Reconstruct the date strings from the components with proper formatting.
start_date_str = f"{start_year}-{start_month:02d}-{start_day:02d}"
end_date_str = f"{end_year}-{end_month:02d}-{end_day:02d}"

# This list will hold all the rows that match the date criteria
processed_rows = []

# Use a try-except block to handle potential file errors
try:
    # Open and read the input file line by line
    with open(input_filename, 'r') as f:
        for line in f:
            # Skip any empty or malformed lines to prevent errors
            if not line.strip():
                continue

            # Split the line into its different data components
            data_list = line.strip().strip(',').split(',')

            # Extract the date string from the line
            line_date_str = data_list[0]
            # Convert the string to a datetime object to allow for comparison
            line_date = datetime.strptime(line_date_str, '%Y-%m-%d').date()

            # Check if the line's date is within the desired range (inclusive)
            if start_date <= line_date <= end_date:
                # If the date matches, process the rest of the line
                time = data_list[1]
                unit = data_list[2].split(':')[1].strip()

                # Create a dictionary to hold the data for this row
                row_dict = {
                    'Date': line_date_str,
                    'Time': time,
                    'Unit': unit
                }

                # Loop through the sensor readings (S1, S2, etc.)
                for item in data_list[3:]:
                    key, value = item.split(':')
                    # Add each sensor reading to the dictionary
                    row_dict[key.strip()] = float(value.strip())

                # Append the processed dictionary to our list of rows
                processed_rows.append(row_dict)

    # After checking all lines, see if we found any matching rows
    if processed_rows:
        # Create the final DataFrame from the list of dictionaries
        final_df = pd.DataFrame(processed_rows)

        # Save the final DataFrame to a CSV file, without the pandas index
        final_df.to_csv(output_filename, index=False)
        print(f"Processing complete. Data between {start_date_str} and {end_date_str} has been saved to '{output_filename}'.")
    else:
        print("No data was found within the specified date range.")

except FileNotFoundError:
    print(f"Error: The input file '{input_filename}' was not found. Please make sure the file is in the same directory as the script.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [1]:
import pandas as pd
import re

# --- User settings ---
# Define the names for your input and output files.
input_filename = 'DatalogAntiguo.txt'
cleaned_filename = 'datalog_cleaned.txt'
output_filename = 'Datalog.csv'
# ---------------------

# --- Clean the input file ---
try:
    with open(input_filename, 'r') as f_in, open(cleaned_filename, 'w') as f_out:
        for line in f_in:
            if line.strip():
                f_out.write(line)
    print(f"Successfully removed empty lines from '{input_filename}' and saved to '{cleaned_filename}'.")
except FileNotFoundError:
    print(f"Error: The input file '{input_filename}' was not found.")
    exit()
except Exception as e:
    print(f"An error occurred during file cleaning: {e}")
    exit()
# --------------------------

# This list will hold all the rows of data
processed_rows = []

# Use a try-except block to handle potential file errors
try:
    # Open and read the cleaned input file line by line
    with open(cleaned_filename, 'r') as f:
        for line in f:
            # Skip any empty or malformed lines to prevent errors
            if not line.strip():
                continue

            # Use regex to find all sensor readings in the line
            sensor_readings = re.findall(r'(S\d+):\s*(-?\d+\.\d+)\s*C\s*°,?', line)

            # Create a dictionary to hold the data for this row
            row_dict = {}

            # Add each sensor reading to the dictionary
            for key, value in sensor_readings:
                row_dict[key] = float(value)

            # Append the processed dictionary to our list of rows
            if row_dict:
                processed_rows.append(row_dict)

    # After checking all lines, see if we found any matching rows
    if processed_rows:
        # Create the final DataFrame from the list of dictionaries
        final_df = pd.DataFrame(processed_rows)

        # Reorder columns to have S0, S1, S2, etc. in order
        final_df = final_df.reindex(sorted(final_df.columns, key=lambda x: int(x[1:])), axis=1)

        # Save the final DataFrame to a CSV file, without the pandas index
        final_df.to_csv(output_filename, index=False)
        print(f"Processing complete. Data has been saved to '{output_filename}'.")
    else:
        print("No data was found in the input file.")

except FileNotFoundError:
    print(f"Error: The cleaned file '{cleaned_filename}' was not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Successfully removed empty lines from 'DatalogAntiguo.txt' and saved to 'datalog_cleaned.txt'.
Processing complete. Data has been saved to 'Datalog.csv'.


In [9]:
import pandas as pd
import re

# --- User settings ---
# Define the names for your input and output files.
input_filename = '1730.TXT'
cleaned_filename = 'datalog_cleaned_A.txt'
output_filename = 'Datalog.csv_A'
# ---------------------

# --- Clean the input file ---
try:
    with open(input_filename, 'r') as f_in, open(cleaned_filename, 'w') as f_out:
        for line in f_in:
            # Only write lines that are not empty to the cleaned file
            if line.strip():
                f_out.write(line)
    print(f"Successfully removed empty lines from '{input_filename}' and saved to '{cleaned_filename}'.")
except FileNotFoundError:
    print(f"Error: The input file '{input_filename}' was not found.")
    exit() # Stop the script if the input file is missing
except Exception as e:
    print(f"An error occurred during file cleaning: {e}")
    exit() # Stop the script on other cleaning errors
# --------------------------

# This list will hold all the rows for the final CSV
processed_rows = []
# Variables to hold the last seen date and time
current_date = None
current_time = None

# Use a try-except block to handle potential file errors
try:
    # Open and read the cleaned input file line by line
    with open(cleaned_filename, 'r') as f:
        for line in f:
            # --- Timestamp Handling ---
            # Check if the line contains a full date and time stamp
            # Regex to find a date (YYYY-MM-DD) and a time (HH:MM:SS)
            timestamp_match = re.search(r'(\d{4}-\d{2}-\d{2}),(\d{2}:\d{2}:\d{2})', line)
            if timestamp_match:
                # If a timestamp is found, update the current date and time for subsequent lines
                current_date = timestamp_match.group(1)
                current_time = timestamp_match.group(2)

            # --- Data Extraction ---
            # Use regex to find all sensor readings in the current line
            # This looks for patterns like "S#: ##.# C°"
            sensor_readings = re.findall(r'(S\d+):\s*(-?\d+\.\d+)\s*C\s*°?', line)

            # --- Row Creation Logic ---
            # A row is only created if:
            # 1. A timestamp has already been found in the file (current_date is not None)
            # 2. The current line actually contains sensor data (sensor_readings is not empty)
            if current_date and sensor_readings:
                # Create a dictionary to hold the data for this row
                row_dict = {
                    'Date': current_date,
                    'Time': current_time,
                    'Unit': 'C°' # Set the unit to C° as it's consistent
                }

                # Add each sensor reading found on this line to the dictionary
                for key, value in sensor_readings:
                    row_dict[key] = float(value)

                # Append the completed dictionary to our list of rows
                processed_rows.append(row_dict)

    # --- DataFrame Creation and Export ---
    # After checking all lines, see if we found any valid data
    if processed_rows:
        # Create the final DataFrame from the list of dictionaries
        final_df = pd.DataFrame(processed_rows)

        # Define the desired column order for the CSV
        # Get all columns that start with 'S' and sort them numerically
        sensor_columns = sorted([col for col in final_df.columns if col.startswith('S')], key=lambda x: int(x[1:]))
        column_order = ['Date', 'Time', 'Unit'] + sensor_columns

        # Reorder the DataFrame columns and fill missing sensor values with empty strings
        final_df = final_df.reindex(columns=column_order).fillna('')

        # Save the final DataFrame to a CSV file, without the pandas index
        final_df.to_csv(output_filename, index=False)
        print(f"Processing complete. Data has been saved to '{output_filename}'.")
    else:
        print("No processable data was found in the input file.")

except FileNotFoundError:
    print(f"Error: The cleaned file '{cleaned_filename}' was not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Successfully removed empty lines from '1730.TXT' and saved to 'datalog_cleaned_A.txt'.
No processable data was found in the input file.


In [11]:
import pandas as pd
import re

# --- User settings ---
# Define the names for your input and output files.
input_filename = '1730.TXT'
cleaned_filename = 'datalog_cleaned_A.txt'
output_filename = 'Datalog_A.csv'
# ---------------------
# This list will hold all the rows for the final CSV
processed_rows = []

try:
    with open(input_filename, 'r') as f:
        for line in f:
            # --- Skip Header Line ---
            # If the line is empty or a header, skip it and continue to the next one
            if not line.strip() or 'Inicio de registro' in line:
                continue

            # --- Data Extraction ---
            # Split the line by commas to separate the parts
            parts = line.strip().split(',')

            # Extract date and time from the first two parts
            date_str = parts[0]
            time_str = parts[1]

            # Create a dictionary for the row with the basic information
            row_dict = {
                'Date': date_str,
                'Time': time_str,
                'Unit': 'C°' # The unit is consistent, so we can set it directly
            }

            # --- Sensor Reading Extraction ---
            # The rest of the line contains the sensor data
            # We re-join the remaining parts in case a comma was in a value
            sensor_data_string = ','.join(parts[2:])

            # Use regex to find all sensor readings (e.g., "S1: 20.38")
            sensor_readings = re.findall(r'(S\d+):\s*(-?\d+\.?\d*)', sensor_data_string)

            # Add each found sensor reading to our dictionary
            for key, value in sensor_readings:
                row_dict[key] = float(value)

            # Add the completed row to our list
            processed_rows.append(row_dict)


    # --- DataFrame Creation and Export ---
    if processed_rows:
        # Create the final DataFrame from our list of row dictionaries
        final_df = pd.DataFrame(processed_rows)

        # Get all columns that start with 'S' and sort them numerically for clean ordering
        sensor_columns = sorted([col for col in final_df.columns if col.startswith('S')], key=lambda x: int(x[1:]))
        
        # Define the final column order
        column_order = ['Date', 'Time', 'Unit'] + sensor_columns

        # Reorder the DataFrame and fill any missing sensor values with an empty string
        final_df = final_df.reindex(columns=column_order).fillna('')

        # Save the DataFrame to a CSV file
        final_df.to_csv(output_filename, index=False)
        print(f"Processing complete. Data from '{input_filename}' has been saved to '{output_filename}'.")
    else:
        print(f"No processable data was found in '{input_filename}'.")

except FileNotFoundError:
    print(f"Error: The input file '{input_filename}' was not found. Please make sure it is in the same directory.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Processing complete. Data from '1730.TXT' has been saved to 'Datalog_A.csv'.
